# Selenium 구글 이미지 스크래핑

## Selenium 및 웹 드라이버 설치

In [ ]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-div-shm-usage')

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [1,418 kB]
Get:15 http:

## 필요 라이브러리 import

In [ ]:
import os
import time
import socket

from urllib.request import urlretrieve
from urllib.error import HTTPError, URLError
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException
from PIL import Image

## `scroll_down()`: 스크롤을 내리는 함수

In [ ]:
def scroll_down():
  scroll_count = 0
  print("[scroll_down(): 스크롤 다운 시작")

  last_height = wd.execute_script("return document.body.scrollHeight") #자바스크립트로 scrolllHeight 값 반환
  after_click = False

  while True:
    print(f"[스크롤 다운: {scroll_count}]")
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);") #0부터 최대인 scrolllHeight까지 스크롤 다운
    scroll_count += 1
    time.sleep(1)

    new_height = wd.execute_script("return document.body.scrollHeight") #스크롤 다운 후 현재 scrollHeight 값 반환

    if last_height == new_height: #스크롤 다운이 되지 않은 경우 = 마지막까지 스크롤 다운한 경우
      if after_click is True: #결과 더보기 버튼 클릭한 경우 = 끝까지 스크롤 다 내린 경우
        break
      else:
        try:
          more_button = wd.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div/div[5]/input') #결과 더보기 버튼
          if more_button.is_displayed():
            more_button.click()
            after_click = True
        except NoSuchElementException as e:
          print(e) #결과 더보기 버튼 없는 경우 에러 메세지 출력
          break

    last_height = new_height


## `click_and_save()`: 썸네일 이미지 선택 후 원본 이미지 저장

In [ ]:
def click_and_save(dir_name, index, img, img_list_length):
  global scraped_count

  try:
    img.click()
    wd.implicitly_wait(3) #3초 기다림
    src = wd.find_element_by_xpath('//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div[1]/div[1]/div/div[2]/a/img').get_attribute('src')
    if src.split('.')[-1] == 'png':
      urlretrieve(src, dir_name + '/' + str(scraped_count + 1) + ".png")
      print(f" {index+1}/{img_list_length} PNG 이미지 저장")
    else:
      urlretrieve(src, dir_name + '/' + str(scraped_count + 1) + ".jpg")
      print(f" {index+1}/{img_list_length} JPG 이미지 저장")

    scraped_count += 1

  except HTTPError as e:
    print(e)
    pass #무시
  except ElementClickInterceptedException as e:
    print(e)
    pass


## `scraping()`: 구글 이미지 스크래핑 시작

In [ ]:
def scraping(dir_name, query):
  global scraped_count

  url = f"https://www.google.co.kr/search?q={query}&tbm=isch&hl=ko&hl=ko&tbs=isz%3Al&sa=X&ved=0CAEQpwVqFwoTCPjwxLqkvesCFQAAAAAdAAAAABAC&biw=1263&bih=610"
  wd.get(url)
  wd.maximize_window() #윈도우창 최대로
  
  scroll_down() #함수 호출

  div = wd.find_element_by_xpath('//*[@id="islrg"]/div[1]')
  img_list = div.find_elements_by_css_selector('div.bRMDJf.islir > img')
  print(img_list)
  for index, img in enumerate(img_list):
    try:
      click_and_save(dir_name, index, img, len(img_list)) #함수 호출
    except ElementClickInterceptedException as e:
      print(e)
      wd.execute_script("window.scrollTo(0, window.scrollY + 100)")
      time.sleep(1)
      click_and_save(dir_name, index, img, len(img_list))
    except NoSuchElementException as e:
      print(e)
      wd.execute_script("window.scrollTo(0, window.scrollY + 100)")
      time.sleep(1)
      click_and_save(dir_name, index, img, len(img_list))
    except ConnectionResetError as e:
      print(e)
      pass
    except URLError as e:
      print(e)
      pass
    except socket.timeout as e: #통신오류. 기다려도 안 되는 경우
      print(e)
      pass
    except socket.gaierror as e: #gai(get address info). address info 얻지 못하는 경우
      print(e)
      pass
    except ElementNotInteractableException as e:
      print(e)
      break
  
  try:
    print("스크래핑 종료 (성공률: %.2f%%)" % (scraped_count / len(img_list) * 100.0))
  except ZeroDivisionError as e:
    print(e)

  wd.quit()

## `filter_and_remove()`: 일정 해상도 이하이거나 손상된 이미지 제거

In [ ]:
def filter_and_remove(dir_name, query, filter_size):
  filtered_count = 0

  for index, file_name in enumerate(os.listdir(dir_name)): #스크래핑된 파일들
    try:
      file_path = os.path.join(dir_name, file_name) #full path
      img = Image.open(file_path)

      if img.width < filter_size and img.height < filter_size:
        img.close()
        os.remove(file_path)
        print(f"{index} 이미지 제거")
        filtered_count += 1
    except OSError as e:
      print(e)
      os.remove(file_path) #손상된 파일 삭제
      filtered_count += 1
    
  print(f"[이미지 제거 개수: {filtered_count}/{scraped_count}]")

## 이미지 확인

In [ ]:
socket.setdefaulttimeout(30) #너무 오래 걸릴 경우 대비해 timeout 최대 30으로 지정

wd = webdriver.Chrome('chromedriver', options=chrome_options)
scraped_count = 0

path = "./" #현재 디렉토리
query = input("검색어 입력: ")

dir_name = path + query
os.makedirs(dir_name) #이미지 저장할 디렉토리 생성
print(f"[{dir_name} 디렉토리 생성]")

scraping(dir_name, query) #스크래핑 함수 호출
filter_and_remove(dir_name, query, 400) #함수 호출

검색어 입력: sea
[./sea 디렉토리 생성]
[scroll_down(): 스크롤 다운 시작
[스크롤 다운: 0]
[스크롤 다운: 1]
[스크롤 다운: 2]
[스크롤 다운: 3]
[스크롤 다운: 4]
[스크롤 다운: 5]
[스크롤 다운: 6]
[스크롤 다운: 7]
[스크롤 다운: 8]
[스크롤 다운: 9]
[스크롤 다운: 10]
[스크롤 다운: 11]
[<selenium.webdriver.remote.webelement.WebElement (session="9ada754edfaf519f17f3d86ca46fc4b1", element="8be00958-5d68-4f4e-ba25-19c239baaeff")>, <selenium.webdriver.remote.webelement.WebElement (session="9ada754edfaf519f17f3d86ca46fc4b1", element="2aad34cd-cb1f-4fbf-b46b-5f9539ad4b7f")>, <selenium.webdriver.remote.webelement.WebElement (session="9ada754edfaf519f17f3d86ca46fc4b1", element="09c0617a-6ee6-478d-a85f-8ccb648d4b0f")>, <selenium.webdriver.remote.webelement.WebElement (session="9ada754edfaf519f17f3d86ca46fc4b1", element="c6de834c-f5d3-4770-81b5-1a861aaeda80")>, <selenium.webdriver.remote.webelement.WebElement (session="9ada754edfaf519f17f3d86ca46fc4b1", element="d7337aff-7414-4ecc-8942-6813c64967d6")>, <selenium.webdriver.remote.webelement.WebElement (session="9ada754edfaf519f17f3

In [ ]:
!ls sea

104.jpg  185.jpg  272.jpg  374.jpg  459.jpg  557.jpg  663.jpg  735.jpg	798.jpg
118.jpg  188.jpg  288.jpg  378.jpg  45.jpg   560.jpg  66.jpg   736.jpg	801.jpg
128.jpg  197.jpg  294.jpg  384.jpg  460.jpg  561.jpg  670.jpg  73.jpg	804.jpg
130.jpg  198.jpg  295.jpg  389.jpg  461.jpg  573.jpg  675.jpg  740.jpg	806.png
131.jpg  200.jpg  297.jpg  394.jpg  466.jpg  574.jpg  67.jpg   741.png	812.jpg
137.jpg  201.png  300.jpg  395.jpg  469.jpg  587.jpg  682.jpg  745.jpg	813.jpg
141.jpg  204.jpg  316.jpg  397.jpg  470.jpg  591.jpg  683.jpg  747.jpg	814.jpg
145.jpg  206.jpg  317.jpg  398.jpg  476.jpg  597.jpg  685.jpg  748.jpg	818.jpg
146.jpg  216.jpg  319.jpg  402.jpg  480.jpg  598.jpg  686.jpg  74.jpg	826.jpg
150.jpg  228.jpg  31.jpg   405.jpg  488.jpg  59.jpg   696.jpg  757.jpg	827.jpg
159.jpg  230.jpg  320.jpg  410.jpg  489.jpg  601.jpg  69.jpg   773.jpg	833.jpg
160.jpg  231.jpg  337.jpg  418.jpg  491.jpg  60.jpg   702.jpg  781.jpg	90.jpg
161.jpg  233.jpg  340.jpg  419.jpg  510.jpg  622.jpg  7

In [ ]:
from google.colab import files
files.download('./sea/104.jpg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>